<a href="https://colab.research.google.com/github/luram94/OpenFOAM-4.x/blob/master/content_generation_pipeline_dishwasher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vllm
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.1/37.1 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

# Funciones

In [2]:
def clean_raw_text(raw_text):
    while "••" in raw_text:
        raw_text = raw_text.replace("••","•")

    while "  " in raw_text:
        raw_text = raw_text.replace("  "," ")
    while "...." in raw_text:
        raw_text = raw_text.replace("....","...")

    while "''''" in raw_text:
        raw_text = raw_text.replace("''''","'''")

    while "\n \n \n" in raw_text:
        raw_text = raw_text.replace("\n \n \n","\n \n")

    while "\n\n\n" in raw_text:
        raw_text = raw_text.replace("\n\n\n","\n\n")

    return raw_text

In [3]:
from vllm import LLM, SamplingParams
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load Model

In [4]:
# Configuration for the language model
sampling_config = {
    'temperature':0.1,
    'top_p':0.90,
    'top_k':50,
    'max_tokens':10000,
    'repetition_penalty':1.1,
}

sampling_params = SamplingParams(
                                  **sampling_config,
                                  )

model_config = {
                'model':"TheBloke/Mistral-7B-Instruct-v0.2-AWQ",
                'quantization':"awq",
                'dtype':"half",
                'max_seq_len_to_capture':15000,
                'max_model_len':15000,
                'gpu_memory_utilization': 0.9,
}

In [5]:
llm = LLM(
            **model_config,
          )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

WARNING 06-06 10:05:07 config.py:213] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 06-06 10:05:07 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='TheBloke/Mistral-7B-Instruct-v0.2-AWQ', speculative_config=None, tokenizer='TheBloke/Mistral-7B-Instruct-v0.2-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=15000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=TheBloke/Mistral-7B-Instruct-v0.2-AWQ)


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

INFO 06-06 10:05:09 selector.py:120] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 06-06 10:05:09 selector.py:51] Using XFormers backend.
INFO 06-06 10:05:13 selector.py:120] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 06-06 10:05:13 selector.py:51] Using XFormers backend.
INFO 06-06 10:05:13 weight_utils.py:207] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

INFO 06-06 10:05:37 weight_utils.py:250] No model.safetensors.index.json found in remote.
INFO 06-06 10:05:38 model_runner.py:146] Loading model weights took 3.8814 GB
INFO 06-06 10:05:48 gpu_executor.py:83] # GPU blocks: 3384, # CPU blocks: 2048
INFO 06-06 10:05:51 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-06 10:05:51 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-06 10:06:06 model_runner.py:924] Graph capturing finished in 14 secs.


In [ ]:
diciconario = {"lista": []}

In [ ]:
diciconario["lista"].append("hola")
diciconario

{'lista': ['hola']}

In [8]:
texto_raw  = """Caution: Exercise care when using the removal tool to
eject the internal SIM card/Memory card tray.
1. Insert the removal tool into the hole on the
SIM card/Memory card tray, and then push until
the tray pops out.
2. Place the SIM card on the tray.
• Make sure that the card’s gold contacts face
down into the device and that the card is
positioned as shown.
3. Place the memory card on the tray with the gold
contacts facing down.
4. Slide the SIM card/Memory card tray back into
the slot.
Important: To prevent water and dust from entering the
device, remove any dust on the O-Ring and ensure that
the SIM card tray is closed tightly.
Caution: If the SIM card is not inserted correctly, the
device cannot detect it. Reorient the card and place it
back into the slot if the SIM is not detected.
Getting Started 8 Assemble Your Device
Maintaining Water and
Dust Resistance
Note: Water-resistant and dustproof based on
IP68 rating, which tests submersion up to
5.0 feet for up to 30 minutes.
This device is rated IP68 using the Ingress Protection
rating system.
Your device has been tested in a controlled
environment and shown to be water and dust resistant
in certain circumstances (meets requirements of
classification IP68 as described by the international
standard IEC 60529 - Degrees of Protection provided
by Enclosures [IP Code]; test conditions: 15-35°C,
86-106 kPa, 5 feet, for 30 minutes). Despite this
classification, your device is not impervious to
water damage in any situation. It is important that all
compartments are closed tightly.
Follow these tips carefully to prevent damage to the
device.
• Whenever your device gets wet, dry it thoroughly
with a clean, soft cloth.
• Do not expose the device to salt water or ionized
water, or to water over 5.0 feet deep for more than
30 minutes. If your device is exposed to fresh
water, dry it thoroughly with a clean, soft cloth. If
the device is exposed to any liquid other than fresh
water, rinse the device with fresh water immediately
and dry it thoroughly with a clean, soft cloth.
Failure to rinse the device in fresh water and dry it
as instructed may cause the device to suffer from
operability or cosmetic issues.
• Do not expose the device to water at high pressure,
such as ocean waves or a waterfall.
• If the device is dropped or receives an impact, the
water and dust resistant features of the device may
be damaged.
• The touchscreen and other features may not work
properly if the device is used in water or in other
liquids.
Getting Started 9 Assemble Your Device
Charge the Battery
Your device is powered by a rechargeable, standard
Li-Ion battery. A USB charger (charging head and
USB cable) is included with the device for charging
the battery from any standard outlet.
The battery comes partially charged. A fully
discharged battery requires up to four hours of charge
time. You can use the device while charging.
Note: It is recommended you fully charge the battery
before using your device for the first time.
Note: For fastest charging times, use the Adaptive
Fast Charging charger that came with your device. This
charger provides faster charging only when connected to
devices that support Adaptive Fast Charging.
Caution: Use only charging devices and batteries
approved by Samsung. Samsung accessories
are designed to maximize battery life. Using other
accessories may invalidate your warranty and may
cause damage.
1. Insert the USB cable into the USB charger/
Accessory port.
2. Connect the USB cable to the charging head, and
then plug the charging head into a standard outlet.
3. Unplug the charging head from the outlet and
remove the USB cable from the device when
charging is complete.
Note: This device is also compatible with wireless
charging devices approved by Samsung, which are
sold separately.
Getting Started 10 Assemble Your Device
When to Charge the Battery
When the battery is low and only a few minutes of
operation remain, the device sounds a warning tone at
regular intervals.
In this condition, your device conserves its remaining
battery power by dimming the screen.
When the battery level becomes too low, the device
automatically turns off.
For a quick check of your battery level:
► View the battery charge indicator located in the
top right corner of your device’s display. A solid
color ( ) indicates a full charge.
You can also choose to display a percentage
value. Viewing a percentage value can provide a
better idea of the remaining charge on the battery.
For more information, see “Battery” on page 149.
Getting Started 11 Set Up Your Device
Set Up Your Device
Turn Your Device On or Off
The instructions below explain how to turn your device
on or off.
Turn the Device On
To turn the device on:
► Press and hold the Power key until the device
turns on.
Turn the Device Off
To turn the device off:
1. Press and hold the Power key until the
Device options menu is displayed.
2. Tap Power off, and confirm if prompte"""

In [71]:
texto_raw  = """
NOTE: This dishwasher is designed to be enclosed on the top
and both sides by standard residential kitchen cabinetry.
Select a location as close to the sink as possible for easy
access to water supply and drain lines.
For proper dishwasher operation and appearance, ensure
that the enclosure is square and has the dimensions shown
in Figure 1 below.
If the dishwasher is to be installed in a corner, make sure that
there is adequate clearance to open the door. See Figure
2 below.
After locating the proper place for your new dishwasher,
you will need to make any required openings to allow for
passage of the water, drain and electrical line. In order to
avoid interference with the dishwasher when sliding it into the
cabinet, place your openings within the dimensions shown
in Figure 3 below.
Required Openings:
4
1
/
4
” x 2” (108 x 52mm) - To pass the included electrical
supply junction box through to
an adjacent cabinet
4” x 2” (100 x 50mm) - To pass the included water
supply line toward the water
supply
1
1
/
4
” (32mm) diameter - To pass the dishwasher
drain hose toward the drain
connection
NOTE: If the incoming electric supply, water supply and
drain connections are all in the same cabinet, the one
4
1
/
4
” x 2” (108 x 52mm) hole will be large enough for all three
to pass through.
Check clearance
between
dishwasher door
and wall
Countertop
WARNING
Avoid Scalding or Electrical Shock Hazard!
Make sure the water supply and electrical supply are
shut off before installation or service.
Avoid Electrical Shock/Fire Hazard
Do not allow the electrical and water supply lines
to touch.
WARNING
4-1/2"
(114mm)
2-1/2"
(64mm)
90° 24” (61cm)
2” (5cm)
4”(10cm)
34”(86.5cm)
24”-24¼” (61-61.6 cm)
Before sliding the dishwasher into the cabinet, remove the
hose clip at the back of the dishwasher, as shown below
and discard.
7
Electrical Supply
The customer has the responsibility of ensuring that the
dishwasher electrical installation is in compliance with all
national and local electrical codes and ordinances. The
dishwasher is designed for an electrical supply of 120V,
60 Hz, AC, connected to a dishwasher-dedicated, properly
grounded electrical circuit with a fuse or breaker rated for 15
amps. Electrical supply conductors shall be a minimum #14
AWG copper wire rated at 75°C (167°F) or higher.
Grounding Instructions
The dishwasher must be properly grounded before
operating. This appliance must be connected to
a grounded metal permanent wiring system, or an
equipment grounding conductor must be run with the circuit
conductors and connected to the equipment grounding
terminal or lead on the dishwasher. Make sure that the
dishwasher is connected to a suitable ground in compliance
with all local codes or, in the absence of a local code, with
the NATIONAL ELECTRICAL CODE in the United States or
the CANADIAN ELECTRIC CODE C22.1-latest edition in
Canada as well as any provincial/state or municipal or local
codes that apply.
Electrical Connection
The dishwasher electrical supply junction box and dedicated
receptacle must be mounted in an accessible cabinet
adjacent to the dishwasher (do not mount the junction box
or receptacle behind the dishwasher). You will need a
4
1
/
4
” x 2” (108 x 51mm) opening throught the cabinet in
order to pass the junction box through (see Figure 4). If the
opening is made through wood, sand it smooth. If the opening
is made through metal, use the included protective grommet
or other approved method to protect wiring from damage.
The electrical supply can
be connected in two ways:
Method A - Three prong plug and receptacle
Use the included three-prong plug and junction box to
connect to a dedicated household receptacle. Make sure
the household receptacle meets the electrical supply
requirements as well as national and local codes.
ELECTRICAL PREPARATION

Avoid Electrical Shock Hazard
Do not work on an energized circuit. Doing so
could result in serious injury or death. Only
qualiied electricians should perform electrical
work. Do not attempt any work on the dishwasher
electric supply circuit until you are certain the
circuit is de-energized.
Avoid Fire Hazard
Make sure electrical work is properly installed.
Only qualiied electricians should perform
electrical work.
WARNING
WARNING
Avoid Fire Hazard
Make sure there are no loose electrical connections.
Make sure all electrical connections are properly
made.
WARNING
Dishwasher Electrical Rating
Volts Hertz Amperes Watts
120 60 15 1,450
(max) 12
Dedicated
Receptacle or
Field Wiring
Dishwasher
Electrical Supply
Junction Box
Electric cord with
junction box and
three prong plug
are included
with dishwasher
Figure 4
Figure 5
CLICK
Electrical Preparation
300
8
Hot Water Supply
The hot water heater should be set to deliver approximately
120° F (49° C) water to the dishwasher. Water that is too
hot can cause some detergents to lose effectiveness. Lower
water temperatures will increase run times. The hot water
supply pressure must be between 15 - 145 psi (1 - 10 bar).
IMPORTANT NOTES:
· If using a solder joint instead of a compression itting, be
sure to make all solder connections before connecting the
water supply line to the dishwasher.
· Make sure there are no sharp bends or kinks in the water
line that might restrict water low.
· Always use the appropriate seal when making plumbing
connections.
· Before connecting the water supply line to the dishwasher,
lush the incoming water line for approximately 5 minutes
to clear any foreign material.
· Turn on the water supply and check for leaks after
connections are made.
To permanently connect to household or ield wiring:
1. Remove the dishwasher electrical supply junction box
cover and connect to the power supply cord from the
house installation. (see Figure 5).
2. Remove 2” to 3” (51 - 76mm) of the outer casing of the
household or ield supply wiring as shown in Figure 6.
Remove 3/8” to 1/2” (10 - 13mm) of the insulation from
each wire as shown in Figure 6.
3. Insert the bare copper or green wire(ground) to the
“G” ground connection “ “ of the terminal block
and securely tighten the terminal block screw (see
Figure 5).
4. Insert the white (neutral) wire to the “N” connection of
the terminal block and securely tighten the terminal block
screw.
5. Insert the black(hot) wire to the “L” connection of the
terminal block and securely tighten the terminal block
screw.
6. Check all electrical connections to make sure they are
secure and then attach the junction box cover with the 4
screws.
Avoid Electrical Shock Hazard
To avoid possible injury or property damage,
care should be exercised when the dishwasher is
installed or removed to reduce the likelihood of
damage to the power cord.
WARNING
Temperatures required for soldering and sweating
will damage the dishwasher. If plumbing lines
are to be soldered or sweated, keep the heat
source at least 6 inches (152.4 mm) away from
the dishwasher"""

In [14]:
type(texto_raw)

str

In [73]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=200,
        separators=["\n\n", ".\n", "\n", ". ", " ", ""]
        # separators=["\n\n", "\.\n", "\n", "(?<=\. )", " ", ""], esto es regex
    )


In [74]:
texts = text_splitter.split_text(clean_raw_text(texto_raw))

In [75]:
texts

['NOTE: This dishwasher is designed to be enclosed on the top \nand both sides by standard residential kitchen cabinetry. \nSelect a location as close to the sink as possible for easy \naccess to water supply and drain lines. \nFor proper dishwasher operation and appearance, ensure \nthat the enclosure is square and has the dimensions shown \nin Figure 1 below.\nIf the dishwasher is to be installed in a corner, make sure that \nthere is adequate clearance to open the door. See Figure \n2 below',
 '.\nIf the dishwasher is to be installed in a corner, make sure that \nthere is adequate clearance to open the door. See Figure \n2 below.\nAfter locating the proper place for your new dishwasher, \nyou will need to make any required openings to allow for \npassage of the water, drain and electrical line. In order to \navoid interference with the dishwasher when sliding it into the \ncabinet, place your openings within the dimensions shown \nin Figure 3 below',
 '.\nRequired Openings:\n4\n1\n/

In [76]:
content_generation_dict = {"texts": texts}

In [77]:
def subject_identification_prompt_template(user_query, context):
    prompt = f"""
<|system|>
You are a an assistant who has to identify if the context in "Context:" is related to the topic the user is writing about in "User_query:". You ONLY can answer "True" or "False" without spaces punctuation marks or anything.</s>
Example of your response. "Response:True"</s>
Context: {context}</s>
<|user|>
User_query: {user_query}</s>
<|assistant|></s>
Response:
"""
    return prompt


In [78]:
user_query = "Create for me training content about the muscular"
subject_identification_prompt = [subject_identification_prompt_template(user_query, context) for context in texts]

In [79]:
subject_mask = []
for idx, output in enumerate(llm.generate(subject_identification_prompt, sampling_params)):
  print(f"Index: {idx}")
  subject_mask.append(output.outputs[0].text.strip().split('.')[0])
  print(output.outputs[0].text + "\n\n")


Processed prompts:  96%|█████████▌| 22/23 [00:06<00:00,  3.58it/s, Generation Speed: 44.64 toks/s]

KeyboardInterrupt: 

In [26]:
subject_mask

['True',
 'True',
 'False',
 'False',
 'True',
 'True',
 "True\n\nHowever, it's important to note that while the context does mention the electrical installation of a dishwasher, the user query specifically asks for creating training content about installing a dishwasher, not just the electrical aspect",
 "True\n\nContext is related to the installation of a dishwasher, which is the topic of the user's query",
 'True',
 'False',
 'True',
 'False',
 'False',
 'False',
 'True',
 'True',
 'True',
 'True',
 'False',
 'False',
 'True',
 'False']

In [28]:
texts[2]

'.\nRequired Openings:\n4\n1\n/\n4\n” x 2” (108 x 52mm) - To pass the included electrical \nsupply junction box through to \nan adjacent cabinet\n4” x 2” (100 x 50mm) - To pass the included water \nsupply line toward the water \nsupply\n1\n1\n/\n4\n” (32mm) diameter - To pass the dishwasher \ndrain hose toward the drain \nconnection\nNOTE: If the incoming electric supply, water supply and \ndrain connections are all in the same cabinet, the one \n4\n1\n/\n4\n” x 2” (108 x 52mm) hole will be large enough for all three'

In [29]:
subject_mask_bool = [s == "True" for s in subject_mask]
subject_mask_bool = ["True" in subject_mask_elem for subject_mask_elem in subject_mask]

In [31]:
content_generation_dict['context_to_use_raw'] = [context  for context, boolean in zip(content_generation_dict['texts'], subject_mask_bool) if boolean]

In [33]:
for idx, text in enumerate(content_generation_dict['context_to_use_raw']):
  print(f"Index: {idx}")
  print(text+"\n\n\n")

Index: 0
NOTE: This dishwasher is designed to be enclosed on the top 
and both sides by standard residential kitchen cabinetry. 
Select a location as close to the sink as possible for easy 
access to water supply and drain lines. 
For proper dishwasher operation and appearance, ensure 
that the enclosure is square and has the dimensions shown 
in Figure 1 below.
If the dishwasher is to be installed in a corner, make sure that 
there is adequate clearance to open the door. See Figure 
2 below



Index: 1
.
If the dishwasher is to be installed in a corner, make sure that 
there is adequate clearance to open the door. See Figure 
2 below.
After locating the proper place for your new dishwasher, 
you will need to make any required openings to allow for 
passage of the water, drain and electrical line. In order to 
avoid interference with the dishwasher when sliding it into the 
cabinet, place your openings within the dimensions shown 
in Figure 3 below



Index: 2
.
Check clearance 
betwee

In [34]:
def content_reduction_prompt_template(context):
    prompt = f"""
<|system|>
You are a helful assistant that must reduce the content inside the following "Context:" as much as possible without loss of information</s>
In order to reduce as much as possible, it does not have to be understandable by humans only have to be understandable by you.</s>
Context: {context}</s>
<|assistant|></s>
Content_reduced:
"""
    return prompt

In [35]:
content_reduction_prompt = [content_reduction_prompt_template(context) for context in content_generation_dict['context_to_use_raw']]

In [36]:
content_reduced_list = []
for idx, output in enumerate(llm.generate(content_reduction_prompt, sampling_params)):
  generation_result = output.outputs[0].text
  print(f"Index: {idx}")
  print(f"Raw input:\n {content_generation_dict['context_to_use_raw'][idx]}\n\n")
  print(f"Content reduction result:\n {generation_result}\n\n")
  content_reduced_list.append(generation_result)

content_generation_dict['content_reduced'] = content_reduced_list

Processed prompts: 100%|██████████| 13/13 [00:05<00:00,  2.22it/s, Generation Speed: 126.69 toks/s]

Index: 0
Raw input:
 NOTE: This dishwasher is designed to be enclosed on the top 
and both sides by standard residential kitchen cabinetry. 
Select a location as close to the sink as possible for easy 
access to water supply and drain lines. 
For proper dishwasher operation and appearance, ensure 
that the enclosure is square and has the dimensions shown 
in Figure 1 below.
If the dishwasher is to be installed in a corner, make sure that 
there is adequate clearance to open the door. See Figure 
2 below


Content reduction result:
 Dishwasher: Enclose top, sides with cabinets. Install near sink for water, drain access. Ensure square shape, dimensions (Figure 1). Corner installation: sufficient clearance for door opening (Figure 2).


Index: 1
Raw input:
 .
If the dishwasher is to be installed in a corner, make sure that 
there is adequate clearance to open the door. See Figure 
2 below.
After locating the proper place for your new dishwasher, 
you will need to make any required opening

In [38]:
'\n'.join(content_reduced_list)

'Dishwasher: Enclose top, sides with cabinets. Install near sink for water, drain access. Ensure square shape, dimensions (Figure 1). Corner installation: sufficient clearance for door opening (Figure 2).\nInstall dishwasher in corner, ensure sufficient clearance for door opening (Fig. 2). Make openings for water, drain, electrical lines within Fig. 3 dimensions to avoid dishwasher interference.\nVerify dishwasher door-wall clearance.\nCaution: Prevent scalding, electric shock.\nShut off water, electricity for installation, service.\nAvoid electric-water contact. Disconnect 4.5" hose clip from 2.5" dishwasher back, discard. Spacing details: 90°, 24", 2", 4", 34", 24"-24.25". Slide dishwasher in after removal.\nCustomer ensures dishwasher install adheres to elec codes. Dishwasher requires 120V, 60Hz, AC, 15A fused/breakered circuit. Wire min #14 AWG, 75°C rated.\nA 15 amp fused/breakered electrical circuit requires #14 AWG copper wires (min. 75°C rating). Dishwasher needs grounding: con

In [39]:
print(sum(len(s) for s in content_generation_dict['context_to_use_raw']))
print(sum(len(s) for s in content_generation_dict['content_reduced']))

5688
2430


In [40]:
prompt = "develops for me what the following means, knowing that it has been reduced by you'Dishwasher: Enclose top, sides with cabinets. Install near sink for water, drain access. Ensure square shape, dimensions (Figure 1). Corner installation: sufficient clearance for door opening (Figure 2).' "

In [41]:
for idx, output in enumerate(llm.generate(prompt, sampling_params)):
  generation_result = output.outputs[0].text
  print(f"Index: {idx}")
  print(generation_result)

Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.79s/it, Generation Speed: 31.07 toks/s]

Index: 0


This passage describes the requirements for installing a dishwasher in a kitchen. Here is a breakdown of its meaning:

1. Enclose top, sides with cabinets: The dishwasher should be installed next to existing kitchen cabinets on three sides and the top. This design helps to integrate the appliance into the kitchen and make it look more aesthetically pleasing.

2. Install near sink for water, drain access: The dishwasher should be placed close to the kitchen sink to provide easy access to the water supply and drain connections. This arrangement makes it convenient to fill the dishwasher with dishes and water and to connect the appliance to the plumbing system.

3. Ensure square shape, dimensions (Figure 1): The dishwasher should have a square or rectangular shape and specific dimensions to fit properly in the designated space. Figure 1 likely refers to a diagram or illustration that shows the recommended size and shape of the dishwasher.

4. Corner installation: If installing 

In [42]:
joined_list = ['\n\n'.join(content_generation_dict['content_reduced'])]

content_generation_dict['content_reduce_joined'] = joined_list

In [43]:
for elem in joined_list:
  print(elem)
  print("#######################")

Dishwasher: Enclose top, sides with cabinets. Install near sink for water, drain access. Ensure square shape, dimensions (Figure 1). Corner installation: sufficient clearance for door opening (Figure 2).

Install dishwasher in corner, ensure sufficient clearance for door opening (Fig. 2). Make openings for water, drain, electrical lines within Fig. 3 dimensions to avoid dishwasher interference.

Verify dishwasher door-wall clearance.
Caution: Prevent scalding, electric shock.
Shut off water, electricity for installation, service.

Avoid electric-water contact. Disconnect 4.5" hose clip from 2.5" dishwasher back, discard. Spacing details: 90°, 24", 2", 4", 34", 24"-24.25". Slide dishwasher in after removal.

Customer ensures dishwasher install adheres to elec codes. Dishwasher requires 120V, 60Hz, AC, 15A fused/breakered circuit. Wire min #14 AWG, 75°C rated.

A 15 amp fused/breakered electrical circuit requires #14 AWG copper wires (min. 75°C rating). Dishwasher needs grounding: connec

In [57]:
training_objectives = "Create for me training content about how to install the dishwasher and its technical requirements"
pedalogical_objectives = "I need enumerated steps for installing and what I need to have into consideration. Also, I need 2 excersises in order to test my knowledge on these steps"

In [58]:
def content_creation_prompt_template(training_objectives, pedalogical_objectives, context):
    prompt = f"""
<|system|>
You are a an assistant who has to generate training content only based on "User_context_for_content_creation:". You have to take into account "User_training_objectives" and "User_pedalogical_objectives".</s>
The generated content generated in "Generated_content:" have to be in markdown format.</s>
<|user|>
User_training_objectives: {training_objectives}</s>
User_pedalogical_objectives: {pedalogical_objectives}</s>
User_context_for_content_creation: {context}</s>
<|assistant|></s>
Generated_content:
"""
    return prompt

In [59]:
content_creation_prompt = [content_creation_prompt_template(training_objectives, pedalogical_objectives, context) for context in content_generation_dict['content_reduce_joined']]

In [60]:
for idx, output in enumerate(llm.generate(content_creation_prompt, sampling_params)):
  print(f"Index: {idx}")
  subject_mask.append(output.outputs[0].text.strip().split('.')[0])
  print(output.outputs[0].text + "\n\n")

Processed prompts: 100%|██████████| 1/1 [00:20<00:00, 20.64s/it, Generation Speed: 28.15 toks/s]

Index: 0

## Installing a Dishwasher in a Corner: Step by Step Guide

### Before Starting

1. Verify sufficient clearance for door opening (Fig. 2)
2. Shut off water and electricity for installation and service
3. Prepare necessary tools and materials

### Electrical Requirements

1. Ensure a 15 amp fused/breakered electrical circuit (#14 AWG copper wires, min. 75°C rating)
2. Ground the dishwasher properly
   - Connect to permanent wiring system or add equipment grounding conductor
   - Comply with local or US/Canadian electrical codes (NATIONAL ELECTRICAL CODE or C22.1-latest)
3. Provide an accessible cabinet adjacent for electrical connection
   - Opening size: 4.1"x2" for junction box passage
   - Wood: Sand smooth; Metal: Use grommet

### Water Connection

1. Clear water line for 5 minutes before connecting to dishwasher
2. Check for leaks post-connection
3. Connect dishwasher to power supply via house wiring
   - Black wire to "L" in terminal block, tighten screw
   - Check and s

In [6]:
texto_raw = """Machine Learning, Deep
Learning and AI: What’s the
Difference ?
by hdwehle
A lot of different terms bandied about these days when it comes to new data processing
techniques. One person says they’re using machine learning, while another calls it artificial
intelligence. Still others may claim to be doing deep learning, while “cognitive” is the favored
phrase for some. What does it all mean?
While many of these terms are related and can overlap in some ways, there are key differences
that can be important, and that could be a barrier to fully understanding what people mean
when they use these words (assuming they’re using them correctly).
Here’s our quick primer on the origin and the meanings of these words and phrases:
Machine Learning
At its most basic level, machine learning refers to any type of computer program that can
“learn” by itself without having to be explicitly programmed by a human. The phrase (and its
underlying idea) has its origins decades ago – all the way to Alan Turing’s seminal 1950 paper
“Computing Machinery and Intelligence,” which featured a section on his famous “Learning
Machine” that could fool a human into believing that it’s real.
Today, machine learning is a widely used term that encompasses many types of programs that
you’ll run across in big data analytics and data mining. At the end of the day, the “brains”
actually powering most predictive programs – including spam filters, product recommenders,
and fraud detectors — are machine learning algorithms.
Data scientists are expected to be familiar with the differences between supervised machine
learning and unsupervised machine learning — as well as ensemble modeling, which uses a
combination of approaches techniques, and semi-supervised learning, which combines
supervised and unsupervised approaches.
3
In supervised learning, the user trains the program to generate an answer based on a known
and labeled data set. Classification and regression algorithms, including random forests,
decision trees, and support vector machines, are commonly used for supervised learning tasks.
In unsupervised machine learning, the algorithms generate answers on unknown and unlabeled
data. Data scientists commonly use unsupervised techniques for discovering patterns in new
data sets. Clustering algorithms, such as K-means, are often used in unsupervised machine
learning.
Data scientists can program machine learning algorithms using a range of technologies and
languages, including Java, Python, Scala, other others. They can also use pre-built machine
learning frameworks to accelerate the process; Mahout is an example of a machine learning
framework that was popular on Apache Hadoop, while Apache Spark’s MLlib library today has
become a standard.
Deep Learning
Deep learning is a form of machine learning that can utilize either supervised or unsupervised
algorithms, or both. While it’s not necessarily new, deep learning has recently seen a surge in
popularity as a way to accelerate the solution of certain types of difficult computer problems,
most notably in the computer vision and natural language processing (NLP) fields.
Neural networks can have many hidden layers (all_is_magic/Shutterstock) Deep learning is
based on the representation learning (or feature learning) branch of machine learning theory.
By extracting high-level, complex abstractions as data representations through a hierarchical
learning process, deep learning models yield results more quickly than standard machine
learning approaches. In plain English, a deep learning model will learn the features that are
important by itself, instead of requiring the data scientist to manually select the pertinent
features, such as the pointiness of ears found in cat pictures (because it somehow always
comes back to cat pictures in the end).
The “deep” in deep learning comes from the many layers that are built into the deep learning
models, which are typically neural networks. A convolutional neural network (CNN) can be
made up of many, many layers of models, where each layer takes input from the previous layer,
processes it, and outputs it to the next layer, in a daisy-chain fashion.
It was a CNN developed by Google‘s DeepMind team that famously beat the human world
champion of the ancient Chinese game of Go, which many saw as a sign of deep learning’s
ascendance.
4
Deep learning is so popular today due to two main reasons. First it was discovered that CNNs
run much faster on GPUs, such as NVidia‘s Tesla K80 processor. Secondly, data scientists
realized that the huge stockpiles of data we’ve been collecting can serve as a massive training
corpus and thereby supercharge the CNNs into yielding substantial improvement in the
accuracy of computer vision and NLP algorithms. Tensorflow is an example of a software
development framework, created by Google, that is seeing a surge of interest; Caffe, Torch, and
Theano are other examples.
Much of the progress in developing self-driving cars can be attributed to advances in deep
learning using CNNs on GPUs, which has the reciprocal effect of helping to fuel further
advances in deep learning and the broader artificial intelligence field.
Artificial Intelligence
Like machine learning and deep learning, artificial intelligence isn’t “new,” but it’s definitely
experience a renaissance of sorts. And the way people use the word is also changing, much to
the chagrin of traditionalists. When Turing first devised his test, the phrase artificial intelligence
was largely reserved for a technology that could broadly mimic the intelligence of humans. In
that respect, it was a far-off, futuristic thing, like time travel seems to us today. (It took 60
years, but a computer finally passed the Turing Test back in 2014.) Chatbots, such as
Microsoft’s short-lived “Tay,” are the face of AI technology for many.
Today, the phrase artificial intelligence, or just AI, is broadly and generally used to refer to any
sort of machine learning program. In that respect, it’s beginning to supplant “big data” and its
hangers-on, “advanced analytics” and “predictive analytics. For those that hate the term “big
data,” this is probably a good thing.
But some people prefer to reserve the phrase AI for the narrowly defined thing that can
replicate many aspects of human intelligence, and become an entity in its own right. We
haven’t reached that stage, yet, and we may never reach it, although that may not be a bet you
want to take. A year ago, Facebook CEO Mark Zuckerberg predicted we were 5 to 10 years away
from developing an AI that could “actually understand what the content means.”
5
Going ‘Cognitive’
Machine learning, deep learning, and artificial intelligence all have relatively specific meanings,
but are often broadly used to refer to any sort of modern, big-data related processing
approach. In this respect, it’s subject to the inevitable hype that accompanies real
breakthroughs in data processing, which the industry most certainly is enjoying at the moment.
But some in the industry eschew these phrases almost entirely and use their own set of words.
IBM, for instance, refers to its work as cognitive computing. In fact, it went so far as to create a
whole new division of the company called Cognitive Systems; its Power Systems division
actually lives within Cognitive Systems (which invariably will irritate customers who want
nothing but to run their ERP system in peace, thank you very much).
But if you look hard enough, you can find a fairly succinct definition of cognitive on the IBM
website. Big Blue says cognitive systems are “a category of technologies that uses natural
language processing and machine learning to enable people and machines to interact more
naturally to extend and magnify human expertise and cognition.”
So there you go: In IBM’s view, cognitive is the combination of NLP and machine learning, which
makes sense when you consider how IBM is using Watson to not only win at Jeopardy, but also
“read” medical literature. In fact, IBM says Watson has been trained on six types of cancer so
far, and will be trained on eight more this year."""

In [7]:
type(texto_raw)

str

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=200,
        separators=["\n\n", ".\n", "\n", ". ", " ", ""]
        # separators=["\n\n", "\.\n", "\n", "(?<=\. )", " ", ""], esto es regex
    )


In [9]:
texts = text_splitter.split_text(clean_raw_text(texto_raw))

In [10]:
content_generation_dict = {"texts": texts}

In [11]:
def subject_identification_prompt_template(user_query, context):
    prompt = f"""
<|system|>
You are a an assistant who has to identify if the context in "Context:" is related to the topic the user is writing about in "User_query:". You ONLY can answer "True" or "False" without spaces punctuation marks or anything.</s>
Example of your response. "Response:True"</s>
Context: {context}</s>
<|user|>
User_query: {user_query}</s>
<|assistant|></s>
Response:
"""
    return prompt


In [12]:
user_query = "Create for me training content about 'Machine Learning, Deep Learning and AI: What’s the Difference ?'"
subject_identification_prompt = [subject_identification_prompt_template(user_query, context) for context in texts]

In [13]:
subject_mask = []
for idx, output in enumerate(llm.generate(subject_identification_prompt, sampling_params)):
  print(f"Index: {idx}")
  subject_mask.append(output.outputs[0].text.strip().split('.')[0])
  print(output.outputs[0].text + "\n\n")

Processed prompts: 100%|██████████| 27/27 [00:10<00:00,  2.58it/s, Generation Speed: 94.77 toks/s]

Index: 0
True


Index: 1
True. The context discusses terms with overlapping meanings and potential confusion, which aligns with the requested training content about differentiating between Machine Learning, Deep Learning, and AI.


Index: 2
True.
Your context discusses Machine Learning, which aligns with the topic you requested.


Index: 3
False

Context: is related to Artificial Intelligence (AI), specifically the concept of a machine being able to learn and fool humans.

User_query: Create for me training content about 'Machine Learning, Deep Learning and AI: What’s the Difference ?'

The user query is not related to the context as it asks for creating training content about the differences between Machine Learning, Deep Learning and AI, while the context discusses the history and concept of a machine being able to learn and fool humans.


Index: 4
True. The context discusses machine learning and its applications, which is related to the topic of discussing the differences between ma

In [14]:
subject_mask

['True',
 'True',
 'True',
 'False\n\nContext: is related to Artificial Intelligence (AI), specifically the concept of a machine being able to learn and fool humans',
 'True',
 'True',
 'False',
 'False',
 'False',
 'True',
 'True',
 'True',
 'False',
 'True',
 'False',
 'False\n<|system|>\nThe context is not related to the topic of "Machine Learning, Deep Learning and AI: What’s the Difference ?"',
 'False',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'True',
 'False',
 'False\n\nThe context discusses cognitive systems, not specifically Machine Learning, Deep Learning, and AI',
 "False\n\nThe context is about IBM's use of cognitive computing (NLP and machine learning) with Watson, while the user query is asking for training content about the differences between Machine Learning, Deep Learning, and Artificial Intelligence"]

In [15]:
#subject_mask_bool = [s == "True" for s in subject_mask]

subject_mask_bool = ["True" in subject_mask_elem for subject_mask_elem in subject_mask]
subject_mask_bool

[True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False]

In [16]:
content_generation_dict['context_to_use_raw'] = [context  for context, boolean in zip(content_generation_dict['texts'], subject_mask_bool) if boolean]

In [17]:
for idx, text in enumerate(content_generation_dict['context_to_use_raw']):
  print(f"Index: {idx}")
  print(text+"\n\n\n")

Index: 0
Machine Learning, Deep 
Learning and AI: What’s the 
Difference ?
by hdwehle
A lot of different terms bandied about these days when it comes to new data processing 
techniques. One person says they’re using machine learning, while another calls it artificial 
intelligence. Still others may claim to be doing deep learning, while “cognitive” is the favored 
phrase for some. What does it all mean?
While many of these terms are related and can overlap in some ways, there are key differences



Index: 1
phrase for some. What does it all mean?
While many of these terms are related and can overlap in some ways, there are key differences 
that can be important, and that could be a barrier to fully understanding what people mean 
when they use these words (assuming they’re using them correctly)



Index: 2
.
Here’s our quick primer on the origin and the meanings of these words and phrases:
Machine Learning
At its most basic level, machine learning refers to any type of computer program

In [18]:
def content_reduction_prompt_template(context):
    prompt = f"""
<|system|>
You are a helful assistant that must reduce de content inside de following "Context:" as much as possible without loss of information</s>
In order to reduce as much as possible, it does not have to be understandable by humans only have to be understandable by you.</s>
Context: {context}</s>
<|assistant|></s>
Content_reduced:
"""
    return prompt

In [19]:
content_reduction_prompt = [content_reduction_prompt_template(context) for context in content_generation_dict['context_to_use_raw']]

In [20]:
content_reduced_list = []
for idx, output in enumerate(llm.generate(content_reduction_prompt, sampling_params)):
  generation_result = output.outputs[0].text
  print(f"Index: {idx}")
  print(f"Raw input:\n {content_generation_dict['context_to_use_raw'][idx]}\n\n")
  print(f"Content reduction result:\n {generation_result}\n\n")
  content_reduced_list.append(generation_result)

content_generation_dict['content_reduced'] = content_reduced_list

Processed prompts: 100%|██████████| 15/15 [00:07<00:00,  2.07it/s, Generation Speed: 97.50 toks/s]

Index: 0
Raw input:
 Machine Learning, Deep 
Learning and AI: What’s the 
Difference ?
by hdwehle
A lot of different terms bandied about these days when it comes to new data processing 
techniques. One person says they’re using machine learning, while another calls it artificial 
intelligence. Still others may claim to be doing deep learning, while “cognitive” is the favored 
phrase for some. What does it all mean?
While many of these terms are related and can overlap in some ways, there are key differences


Content reduction result:
 Machine_Learning(ML), Deep_Learning(DL) & AI: Term_diff.
ML: Computer_learns_from_data.
DL: ML_with_multi-layered_neural_networks.
AI: Simulates_human_intelligence.
Overlap: ML_can_be_part_of_AI & DL_is_often_used_in_AI.
Key_diff: ML_focuses_on_learning_patterns, AI_on_reasoning_&_perception, DL_on_complex_data.


Index: 1
Raw input:
 phrase for some. What does it all mean?
While many of these terms are related and can overlap in some ways, there are key

In [21]:
'\n'.join(content_reduced_list)

'Machine_Learning(ML), Deep_Learning(DL) & AI: Term_diff.\nML: Computer_learns_from_data.\nDL: ML_with_multi-layered_neural_networks.\nAI: Simulates_human_intelligence.\nOverlap: ML_can_be_part_of_AI & DL_is_often_used_in_AI.\nKey_diff: ML_focuses_on_learning_patterns, AI_on_reasoning_&_perception, DL_on_complex_data.\nSome phrases have distinct meanings despite overlapping or related concepts. Understanding the nuances is crucial.\nMachine learning is a computer method that autonomously learns from data without explicit programming. Originated in Alan Turing\'s 1950 paper "Computing Machinery and Intelligence".\nMachine learning refers to various programs utilized in big data analysis and data mining. Primary functions include spam filtering, product recommendation, and fraud detection, all driven by machine learning algorithms.\nDS should know diff btw supervised ML (SL) & unsupervised ML (UL), ensemble ML (EM), & semi-sup ML (SSM). SL uses labeled data for training. UL uses no label

In [22]:
print(sum(len(s) for s in content_generation_dict['context_to_use_raw']))
print(sum(len(s) for s in content_generation_dict['content_reduced']))

5569
3123


In [23]:
for elem in content_generation_dict['content_reduced']:
  print(elem)
  print("############################")

Machine_Learning(ML), Deep_Learning(DL) & AI: Term_diff.
ML: Computer_learns_from_data.
DL: ML_with_multi-layered_neural_networks.
AI: Simulates_human_intelligence.
Overlap: ML_can_be_part_of_AI & DL_is_often_used_in_AI.
Key_diff: ML_focuses_on_learning_patterns, AI_on_reasoning_&_perception, DL_on_complex_data.
############################
Some phrases have distinct meanings despite overlapping or related concepts. Understanding the nuances is crucial.
############################
Machine learning is a computer method that autonomously learns from data without explicit programming. Originated in Alan Turing's 1950 paper "Computing Machinery and Intelligence".
############################
Machine learning refers to various programs utilized in big data analysis and data mining. Primary functions include spam filtering, product recommendation, and fraud detection, all driven by machine learning algorithms.
############################
DS should know diff btw supervised ML (SL) & unsuper

In [24]:
prompt = "develops for me what the following means, knowing that it has been reduced by you'DS should know diff btw supervised ML (SL) & unsupervised ML (UL), ensemble ML (EM), & semi-sup ML (SSM). SL uses labeled data for training. UL uses no labels. EM combines multiple ML models. SSM mixes SL & UL.' "

In [25]:
for idx, output in enumerate(llm.generate(prompt, sampling_params)):
  generation_result = output.outputs[0].text
  print(f"Index: {idx}")
  print(generation_result)

Processed prompts: 100%|██████████| 1/1 [00:18<00:00, 18.95s/it, Generation Speed: 26.34 toks/s]

Index: 0


I assume you are asking about the difference between supervised learning (SL), unsupervised learning (UL), ensemble learning (EM), and semi-supervised learning (SSM) in machine learning.

1. Supervised Learning (SL): In supervised learning, the model is trained on a labeled dataset, which means that each input example comes with an associated output or target label. The goal of the model is to learn the mapping from inputs to outputs based on this labeled data. Once trained, the model can then be used to predict the output for new, unseen input data. Commonly used algorithms for supervised learning include linear regression, logistic regression, decision trees, random forests, support vector machines, and neural networks.
2. Unsupervised Learning (UL): In unsupervised learning, the model is trained on an unlabeled dataset, which means that there are no target labels provided for the input examples. Instead, the goal of the model is to find patterns or structure in the data w

In [26]:
joined_list = ['\n\n'.join(content_generation_dict['content_reduced'])]

content_generation_dict['content_reduce_joined'] = joined_list

In [27]:
for elem in joined_list:
  print(elem)
  print("#######################")

Machine_Learning(ML), Deep_Learning(DL) & AI: Term_diff.
ML: Computer_learns_from_data.
DL: ML_with_multi-layered_neural_networks.
AI: Simulates_human_intelligence.
Overlap: ML_can_be_part_of_AI & DL_is_often_used_in_AI.
Key_diff: ML_focuses_on_learning_patterns, AI_on_reasoning_&_perception, DL_on_complex_data.

Some phrases have distinct meanings despite overlapping or related concepts. Understanding the nuances is crucial.

Machine learning is a computer method that autonomously learns from data without explicit programming. Originated in Alan Turing's 1950 paper "Computing Machinery and Intelligence".

Machine learning refers to various programs utilized in big data analysis and data mining. Primary functions include spam filtering, product recommendation, and fraud detection, all driven by machine learning algorithms.

DS should know diff btw supervised ML (SL) & unsupervised ML (UL), ensemble ML (EM), & semi-sup ML (SSM). SL uses labeled data for training. UL uses no labels. EM c

In [28]:
training_objectives = "I need my students learn about the concepts Machine learning, Deep Learning and Artificial Intelligence"
pedalogical_objectives = "I need enumerated the main definition of each concept and the main differences between them. Also, I need 2 excersises in order to test the knowledge to difference between these concepts"

In [29]:
def content_creation_prompt_template(training_objectives, pedalogical_objectives, context):
    prompt = f"""
<|system|>
You are a an assistant who has to generate training contect based on "User_context_for_content_creation:". You have to take into account "User_training_objectives" and "User_pedalogical_objectives".</s>
The generated content generated in "Generated_content:" have to be in markdown format.</s>
<|user|>
User_training_objectives: {training_objectives}</s>
User_pedalogical_objectives: {pedalogical_objectives}</s>
User_context_for_content_creation: {context}</s>
<|assistant|></s>
Generated_content:
"""
    return prompt

In [30]:
content_creation_prompt = [content_creation_prompt_template(training_objectives, pedalogical_objectives, context) for context in content_generation_dict['content_reduce_joined']]

In [31]:
for idx, output in enumerate(llm.generate(content_creation_prompt, sampling_params)):
  print(f"Index: {idx}")
  subject_mask.append(output.outputs[0].text.strip().split('.')[0])
  print(output.outputs[0].text + "\n\n")

Processed prompts: 100%|██████████| 1/1 [00:41<00:00, 41.41s/it, Generation Speed: 25.62 toks/s]

Index: 0

## Machine Learning (ML), Deep Learning (DL), and Artificial Intelligence (AI): Term Differences

### Machine Learning (ML)

**Definition:** Machine learning is a computer method that autonomously learns from data without explicit programming. It originated in Alan Turing's 1950 paper "Computing Machinery and Intelligence."

**Functions:** Machine learning is used extensively in big data analysis and data mining. Its primary functions include spam filtering, product recommendation, and fraud detection, all driven by machine learning algorithms.

**Types:** Understand the differences between supervised machine learning (SL), unsupervised machine learning (UL), ensemble machine learning (EM), and semi-supervised machine learning (SSM).

   * **Supervised Learning (SL):** Uses labeled data for training.
   * **Unsupervised Learning (UL):** Uses no labels.
   * **Ensemble Learning (EM):** Combines multiple machine learning models.
   * **Semi-supervised Learning (SSM):** Mixes su

In [39]:
def content_creation_prompt_template(training_objectives, pedalogical_objectives, context):
    prompt = f"""
You are an expert assistant tasked with generating educational content and a quiz based on the user's inputs.

### User Inputs:
- **Training Objectives:** {training_objectives}
- **Pedagogical Objectives:** {pedalogical_objectives}
- **Context for Content Creation:** {context}

### Instructions:
1. **Content Creation:**
   - Provide a structured, comprehensive explanation of the topic(s) considering the training and pedagogical objectives.
   - Use markdown format with clear headings, subheadings, bullet points, and examples.
2. **Exercise Creation:**
   - Develop exercises to assess understanding of the generated content considering the pedalogical objectives.
   - Use various types of questions (e.g., multiple-choice, true/false, short answer).
   - **Vary the correct answers among the options (e.g., not always option A).**
   - Provide the correct answers


### Generated Content:
"""
    return prompt

In [41]:
training_objectives = "I need my students learn about the concepts Machine learning, Deep Learning and Artificial Intelligence"
pedalogical_objectives = "I need the main definition of each concept and the main differences between them. Also, I need 2 excercises in order to test the knowledge to difference between these concepts"

In [42]:
content_creation_prompt = [content_creation_prompt_template(training_objectives, pedalogical_objectives, context) for context in content_generation_dict['content_reduce_joined']]

In [43]:
for idx, output in enumerate(llm.generate(content_creation_prompt, sampling_params)):
  print(f"Index: {idx}")
  subject_mask.append(output.outputs[0].text.strip().split('.')[0])
  print(output.outputs[0].text + "\n\n")

Processed prompts: 100%|██████████| 1/1 [00:37<00:00, 37.22s/it, Generation Speed: 25.58 toks/s]

Index: 0
---
# Machine Learning, Deep Learning, and Artificial Intelligence: Term Differences

## Overview
This section discusses the definitions, main differences, and contextual relationships between `Machine Learning` (ML), `Deep Learning` (DL), and `Artificial Intelligence` (AI).

### Machine Learning (ML)
#### Definition
Machine Learning (ML) is a computer method that autonomously learns from data without explicit programming. It originated in Alan Turing's 1950 paper "Computing Machinery and Intelligence".

#### Functions
Primary functions of ML include:
- Spam filtering
- Product recommendation
- Fraud detection
All driven by machine learning algorithms.

#### Subtypes
Understand the differences between various ML subtypes:

##### Supervised Learning (SL)
Supervised Learning (SL) uses labeled data for training.

##### Unsupervised Learning (UL)
Unsupervised Learning (UL) uses no labels.

##### Ensemble Learning (EM)
Ensemble Learning (EM) combines multiple ML models.

##### Semi